In [11]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.optim import SGD

In [2]:
x = [[1,2], [3,4], [5,6], [7,8]]
y = [[3], [7], [11], [15]]

In [3]:
X = torch.tensor(x).float()
Y = torch.tensor(y).float()

In [4]:
class MyDataSet(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x).float().to(device='cuda' if torch.cuda.is_available() else 'cpu')
        self.y = torch.tensor(y).float().to(device='cuda' if torch.cuda.is_available() else 'cpu')

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [5]:
ds = MyDataSet(x, y)

In [6]:
dl = DataLoader(ds, batch_size=2, shuffle=True)

In [7]:
for x,y in dl:
    print(x, y)

tensor([[1., 2.],
        [3., 4.]]) tensor([[3.],
        [7.]])
tensor([[7., 8.],
        [5., 6.]]) tensor([[15.],
        [11.]])


In [8]:
class MyNeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(2,8)
        self.activation = nn.ReLU()
        self.layer2 = nn.Linear(8,1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.activation(x)
        x = self.layer2(x)
        return x

In [12]:
model = MyNeuralNet()
loss_fn = nn.MSELoss()
opt = SGD(model.parameters(), lr=0.001)

In [13]:
losses = []
for _ in range(50): # 50 epochs
    for data in dl: # Itera sobre o DataLoader em batches
        opt.zero_grad() # Reinicia o gradiente
        x1, y1 = data
        loss_value = loss_fn(model(x1), y1) # Calcula a perda
        
        loss_value.backward() # Calcula o gradiente
        
        opt.step() # Atualiza os pesos
        # Salva a perda para plotar depois
        losses.append(loss_value.detach().numpy())